Import all libraries and load in the numpy arrays

In [1]:
import pickle
import time
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import normalize, to_categorical
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.applications.inception_resnet_v2 import InceptionResNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
import os
os.chdir("C:/Users/moham/Desktop/MyDeepLearningProject/")
X_train= pickle.load(open("Datasets/cropped256/X_train.pickle","rb"))
y_train = pickle.load(open("Datasets/cropped256/y_train.pickle","rb"))

Normalize data

In [2]:
X_train=X_train/255.0
y_train = to_categorical(y_train)

In [3]:
datagen = ImageDataGenerator(
    rotation_range=90,
    fill_mode='reflect',
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    validation_split=0.2,
    brightness_range=[0.3,1.2])

In [4]:
datagen.fit(X_train)

In [5]:
from tensorflow.keras.applications.resnet50 import ResNet50
# load model without classifier layers
model = ResNet50(include_top=False, input_shape=X_train.shape[1:])
# add new classifier layers
flat1 = Flatten()(model.layers[-1].output)
Dense1=Dense(100, activation="relu")(flat1)
Dense2=Dense(100, activation='relu')(Dense1)
Dense3=Dense(50, activation='relu')(Dense2)
output = Dense(3, activation='softmax')(Dense3)
# define new model
model = Model(inputs=model.inputs, outputs=output)
# summarize
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 262, 262, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 128, 128, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 128, 128, 64) 256         conv1_conv[0][0]                 
______________________________________________________________________________________________

In [6]:
model.compile(loss="categorical_crossentropy",
             optimizer="adam",
             metrics=['accuracy'])

In [9]:
print(os.getcwd())

C:\Users\moham\Desktop\MyDeepLearningProject


In [7]:
NAME = "Resnet50-RandWeigts-256-256-{}".format(int(time.time()))
EarlyStop=EarlyStopping(patience=10,monitor='val_loss')
checkpoints=ModelCheckpoint(filepath=f'ModelCheckPoints/{NAME}.h5', verbose=1, save_best_only=True)
tensorboard = TensorBoard(log_dir='DeepModels/{}'.format(NAME))
callbacks=[EarlyStop,checkpoints,tensorboard]
history=model.fit(datagen.flow(X_train, y_train, batch_size=20,
         subset='training'),
         validation_data=datagen.flow(X_train, y_train,
         batch_size=20, subset='validation'),
         steps_per_epoch=len(X_train) / 40, epochs=50,callbacks=[callbacks])

c:\users\moham\desktop\mydeeplearningproject\lib\site-packages\tensorflow\python\keras\utils\generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


Epoch 1/50
84/84 [==============================] - 48s 457ms/step - loss: 1.7409 - accuracy: 0.3452 - val_loss: 2.3392 - val_accuracy: 0.3348